In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler #표준화(평균을 기준)
from sklearn.preprocessing import MinMaxScaler   #정규화(정해진 규칙 기준, min-max)
from sklearn.datasets import load_iris

import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
# cross_val_score: 다양한 샘플링을 해서 여러번 테스트 --> 평균치를 구해본다.!
from sklearn.metrics import *

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from matplotlib import rc, font_manager
import matplotlib.font_manager as fm

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:/Windows/Fonts/malgun.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [3]:
# data12 - 기계마모
# data14 - 앙상블voting, 서울시구별영역나누기
# 위 코드들 이용하여 머신러닝 모델 만들기

In [148]:
df=pd.read_csv('국내여행_병합.csv')

In [149]:
df.columns

Index(['Unnamed: 0', 'year', 'target_area1', 'target_area2', 'target_area3',
       'target_area4', 'target_area5', 'target_area6', 'target_area7',
       'target_area8', 'target_area9', 'target_area10', 'target_area11',
       'no_of_place', 'sleepnights_1', 'sleepnights_2', 'sleepnights_3',
       'sleepnights_4', 'sleepnights_5', 'sleepnights_6', 'sleepnights_7',
       'sleepnights_8', 'accommodation_1', 'accommodation_2',
       'accommodation_3', 'number_of_ppl', 'number_of_children',
       'cost_total_trip', 'satisfied', 'sex', 'age', 'month', 'living_area',
       'transportation1', 'transportation2', 'transportation3', 'reason1',
       'reason2', 'reason3', 'relationship1', 'relationship2', 'relationship3',
       'target'],
      dtype='object')

In [152]:
# sleepnights_1의 컬럼명을 sleepnights로 변경 
df.rename(columns={'sleepnights_1':'sleepnights'}, inplace=True)
# relationship1의 컬럼명을 relationship으로 변경
df.rename(columns={'relationship1':'relationship'}, inplace=True)

In [153]:
# unnamed, year, no_of_place, target_area_1~11, sleepnights_2~8, satisfied, relationship2~3 제외
data=df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']]
target=df['target']

In [154]:
X_train, X_test, y_train, y_test=train_test_split(
    data,
    target,
    test_size=0.2,
    random_state=77
)

In [155]:
knn=KNeighborsClassifier()
svm=SVC()
dt=DecisionTreeClassifier()
model_list=[knn, svm, dt]

In [156]:
def print_cv(model, train, target, k):
    scores=cross_val_score(model, train, target, cv=k)
    print('----교차검증(cv)결과 리스트----\n',scores)
    mean_scores=np.mean(scores)
    print('----교차검증(cv)결과 평균----\n',mean_scores)
    return scores, mean_scores 
    
    ## 함수의 결과로 여러개의 데이터를 return할 수 있다.
    ## 튜플로 묶어서 반환하게 된다.(scores, mean_scores)

In [157]:
model_names=['KNeighborsClassifier','SVC','DecisionTreeClassifier']
sc_list=[]
mean_sc_list=[]
for model in model_list:
    sc, mean_sc=print_cv(model, X_train, y_train, 5)
    sc_list.append(sc)
    mean_sc_list.append(mean_sc)
print('***************************************')
print('----전체 모델 리스트----')
print(model_names)
print('----전체 모델의 교차검증 결과 리스트----')
print(sc_list)
print('----전체 모델의 교차검증 결과 평균----')
print(mean_sc_list)

----교차검증(cv)결과 리스트----
 [0.56469166 0.57263923 0.5788835  0.5788835  0.57038835]
----교차검증(cv)결과 평균----
 0.5730972443155016
----교차검증(cv)결과 리스트----
 [0.66021765 0.66101695 0.66383495 0.66140777 0.66383495]
----교차검증(cv)결과 평균----
 0.662062454645432
----교차검증(cv)결과 리스트----
 [0.59008464 0.56658596 0.5934466  0.56067961 0.55703883]
----교차검증(cv)결과 평균----
 0.5735671296498301
***************************************
----전체 모델 리스트----
['KNeighborsClassifier', 'SVC', 'DecisionTreeClassifier']
----전체 모델의 교차검증 결과 리스트----
[array([0.56469166, 0.57263923, 0.5788835 , 0.5788835 , 0.57038835]), array([0.66021765, 0.66101695, 0.66383495, 0.66140777, 0.66383495]), array([0.59008464, 0.56658596, 0.5934466 , 0.56067961, 0.55703883])]
----전체 모델의 교차검증 결과 평균----
[0.5730972443155016, 0.662062454645432, 0.5735671296498301]


## svc

In [158]:
def grid_search(model, params, k):
    gs_model=GridSearchCV(model, params, cv=k)
    ## gs_model은 최적의 파라미터가 적용된 모델의 객체!
    gs_model.fit(X_train, y_train)
    print(gs_model.best_estimator_) ## 최적의 파라미터가 적용된 모델 객체
    print(gs_model.best_score_) ## 최적의 파라미터가 적용된 모델로 학습한 정확도 결과(acc)
    print(gs_model.best_params_) ## 그리드 서치를 통해 찾은 최적의 파라미터
    print(gs_model) ## grid_search 객체

In [159]:
params={ 
    'C': [0.001, 0.01, 0.1, 1, 10, 25],
    'random_state':[5, 50, 99]
}

In [160]:
grid_search(svm, params, 5)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=5,
    shrinking=True, tol=0.001, verbose=False)
0.662060606060606
{'C': 1, 'random_state': 5}
GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 25],
                         'random_state': [5, 50, 99]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)


In [161]:
gs_svm=SVC(C=1, random_state=5)
gs_svm.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=5,
    shrinking=True, tol=0.001, verbose=False)

In [162]:
def test_validation(model, X_test, y_test):
    predict_result=model.predict(X_test)
    acc=accuracy_score(y_test, predict_result)
    print('accuracy >>', acc)
    cf=confusion_matrix(y_test, predict_result)
    print('confusion matrix >>\n', cf)
    ## confusion matrix는 binary classfication인 경우에는 보기가 괜찮으나,
    ## multi class classification인 경우에는 보기가 힘들다.
    ## confusion matrix를 가독성 좋에 볼 수 있는 것이 classification_report
    c_report=classification_report(y_test, predict_result)
    print('classification_report >>\n', c_report)

In [163]:
test_validation(gs_svm, X_test, y_test)

accuracy >> 0.6521317829457365
confusion matrix >>
 [[  2   0   0 126]
 [  0   1   0 169]
 [  0   0   0  64]
 [  0   0   0 670]]
classification_report >>
               precision    recall  f1-score   support

    식도락,맛집투어       1.00      0.02      0.03       128
        엑티비티       1.00      0.01      0.01       170
       축제,관광       0.00      0.00      0.00        64
      힐링,호캉스       0.65      1.00      0.79       670

    accuracy                           0.65      1032
   macro avg       0.66      0.26      0.21      1032
weighted avg       0.71      0.65      0.52      1032



## 설문 결과 predict

In [164]:
predict_col_list=['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']

In [165]:
len(predict_col_list)

18

In [166]:
def find_index(str):
    for i in range(len(predict_col_list)):
        if predict_col_list[i]==str:
            print(str+'의 index >> ',i)

In [179]:
find_index('age')

age의 index >>  8


In [168]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']].head(1)

,sleepnights,accommodation_1,accommodation_2,accommodation_3,number_of_ppl,number_of_children,cost_total_trip,sex,age,month,living_area,transportation1,transportation2,transportation3,reason1,reason2,reason3,relationship
0,2,1,1,0,3,1,276000,2,38,5,11,1,0,0,14,1,11,1


In [169]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']].head().values

array([[     2,      1,      1,      0,      3,      1, 276000,      2,
            38,      5,     11,      1,      0,      0,     14,      1,
            11,      1],
       [     2,      1,      1,      0,      3,      0, 318000,      2,
            48,      5,     11,      1,      0,      0,      1,     11,
             9,      1],
       [     1,     11,      0,      0,      6,      0, 107000,      1,
            19,      6,     11,      6,      0,      0,      4,     14,
             5,      3],
       [     0,      0,      0,      0,      3,      0,  50000,      2,
            55,      6,     11,      2,      0,      0,      1,      5,
             8,      3],
       [     2,      5,      5,      0,      4,      0, 233000,      1,
            29,      2,     11,      1,      0,      0,     13,     14,
             3,      1]], dtype=int64)

In [170]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']][df['target']=='엑티비티'].head().values

array([[     1,     11,      0,      0,      6,      0, 107000,      1,
            19,      6,     11,      6,      0,      0,      4,     14,
             5,      3],
       [     0,      0,      0,      0,      3,      1,  42000,      1,
            37,      6,     11,      1,      0,      0,      8,      1,
             3,      1],
       [     0,      0,      0,      0,      4,      0, 100000,      1,
            68,      8,     11,      1,      0,      0,     10,     11,
            14,      5],
       [     0,      0,      0,      0,      4,      2,  78048,      2,
            36,     12,     11,      1,      0,      0,     10,     14,
             8,      1],
       [     0,      0,      0,      0,      4,      0, 120000,      1,
            16,      8,     11,      6,      0,      0,     11,      0,
             0,      3]], dtype=int64)

In [172]:
gs_svm.predict([[     1,     11,      0,      0,      6,      0, 107000,      1,
            19,      6,     11,      6,      0,      0,      4,     14,
             5,      3]])

array(['엑티비티'], dtype=object)

In [171]:
df[['sleepnights', 'accommodation_1', 'accommodation_2', 'accommodation_3',
        'number_of_ppl', 'number_of_children', 'cost_total_trip', 'sex', 'age', 'month', 'living_area', 
        'transportation1', 'transportation2', 'transportation3', 
         'reason1', 'reason2', 'reason3', 'relationship']][df['target']=='식도락,맛집투어'].head().values

array([[    0,     0,     0,     0,     6,     0, 70000,     1,    30,
           12,    11,     2,     5,     0,     8,     4,     0,     3],
       [    0,     0,     0,     0,     4,     0, 97000,     1,    28,
            5,    11,     5,    10,     0,     5,     9,     3,     3],
       [    0,     0,     0,     0,     5,     1, 45000,     2,    37,
            1,    11,     1,     0,     0,     8,     5,    14,     1],
       [    0,     0,     0,     0,     4,     0, 40000,     1,    48,
            5,    11,     1,     0,     0,     8,    14,     0,     3],
       [    0,     0,     0,     0,     3,     1, 20000,     2,    38,
            6,    11,     1,     0,     0,     1,     8,     4,     1]],
      dtype=int64)

## pickle, joblib

In [173]:
import pickle
import joblib

In [174]:
joblib.dump(gs_svm, 'trip_gs_svm.pkl')

['trip_gs_svm.pkl']

In [175]:
loaded_model = joblib.load('trip_gs_svm.pkl')
loaded_model

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=5,
    shrinking=True, tol=0.001, verbose=False)

In [178]:
loaded_model.predict([[    0,     0,     0,     0,     4,     0, 40000,     1,    48,
            5,    11,     1,     0,     0,     8,    14,     0,     3]])

array(['식도락,맛집투어'], dtype=object)

## 설문지 답변 전처리, Spring에서 get방식으로 Django에 넘겨줌